In [1]:
import pandas as pd
import numpy as np

from datasets import Dataset
from transformers import AutoTokenizer, EsmForSequenceClassification, TrainingArguments, Trainer

/nfs/home/ababjac/base-venv/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-02-03 12:06:41.467673: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-03 12:06:41.703828: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-03 12:06:41.709796: W tensorflow/stream_executor/platform/default/dso_loade

In [2]:
PATH='/lustre/isaac/proj/UTK0196/deep-surface-protein-data/'

In [3]:
df = pd.read_csv(PATH+'M0059E_training_set.tsv', delimiter=',', nrows=5000, header=0)

In [4]:
surf_series = df['surf.sequence']
deep_series = df['deep.sequence']

classification_df = pd.DataFrame({'text' : surf_series.append(deep_series, ignore_index=True), 'label' : [0]*surf_series.size+[1]*deep_series.size})
classification_df

,text,label
0,MTERGKYVCVGLGEILWDMLPEGKQLGGAPANFAYHAQALRGQGVV...,0
1,MTVDGKYLCVGLGEILWDMLPGGKQLGGAPANFAYHSQALGAQGVV...,0
2,MKVALVGLLQSGKSTILASLSGKAIPAIGSASIEEAIVSVPDDRFD...,0
3,MKVALIGLLQSGKSTILASLTGKAIPAIGSASIEETIVPVPDERFD...,0
4,MKVALIGLLQSGKSTILASLTGKAVPAAGSASIEEAIVPVPDERFD...,0
...,...,...
9995,MDKDTVILVVDDEREHADGIAEAMEKLCGRAIAVYNGADALEIVRN...,1
9996,MDKDTVILVVDDEREHADGIAEAMEKLCGRAIAVYNGADALEIVRN...,1
9997,MDKDTVILVVDDEREHADGIAEAMEKLCGRAIAVYNGADALEIVRN...,1
9998,MDKDTVILVVDDEREHADGIAEAMEKLCGRAIAVYNGADALEIVRN...,1


In [5]:
def overlap_sequence(seq, word_length, overlap):
    if overlap >= word_length:
        print('Overlap must be less than word length')
        return
    
    for i in range(0, len(seq)-overlap, word_length-overlap):
        yield seq[i:i+word_length]
        
def get_overlap_array(seq, word_length=5, overlap=2):
    return np.array(list(overlap_sequence(seq, word_length, overlap)))

def get_overlap_string(seq, word_length=5, overlap=2):
    return ' '.join(list(overlap_sequence(seq, word_length, overlap)))

In [6]:
classification_df['text'] = classification_df['text'].transform(get_overlap_string)
classification_df

,text,label
0,MTERG RGKYV YVCVG VGLGE GEILW LWDML MLPEG EGKQ...,0
1,MTVDG DGKYL YLCVG VGLGE GEILW LWDML MLPGG GGKQ...,0
2,MKVAL ALVGL GLLQS QSGKS KSTIL ILASL SLSGK GKAI...,0
3,MKVAL ALIGL GLLQS QSGKS KSTIL ILASL SLTGK GKAI...,0
4,MKVAL ALIGL GLLQS QSGKS KSTIL ILASL SLTGK GKAV...,0
...,...,...
9995,MDKDT DTVIL ILVVD VDDER EREHA HADGI GIAEA EAME...,1
9996,MDKDT DTVIL ILVVD VDDER EREHA HADGI GIAEA EAME...,1
9997,MDKDT DTVIL ILVVD VDDER EREHA HADGI GIAEA EAME...,1
9998,MDKDT DTVIL ILVVD VDDER EREHA HADGI GIAEA EAME...,1


In [7]:
ds = Dataset.from_pandas(classification_df)

In [8]:
tokenizer = AutoTokenizer.from_pretrained("facebook/esm2_t6_8M_UR50D")

In [9]:
tokenizer.is_fast

False

In [10]:
tokenized_ds = ds.map(lambda d : tokenizer(d['text'], return_tensors="pt", padding=True), batched=True)

100%|███████████████████████████████████████████| 10/10 [00:11<00:00,  1.16s/ba]


In [11]:
init_splits = tokenized_ds.train_test_split(test_size=0.2)

tmp = init_splits['train']
test_ds = init_splits['test']

splits = tmp.train_test_split(test_size=0.1)
train_ds = splits['train']
val_ds = splits['test']

In [12]:
model = EsmForSequenceClassification.from_pretrained("facebook/esm2_t6_8M_UR50D")
#logits = model(**inputs).logits
#predicted_class_id = logits.argmax().item()

Some weights of the model checkpoint at facebook/esm2_t6_8M_UR50D were not used when initializing EsmForSequenceClassification: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'esm.contact_head.regression.bias', 'esm.contact_head.regression.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing EsmForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing EsmForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of EsmForSequenceClassification were not initialized from the model checkpoint at facebook/esm2_t6_8M_UR50D and are newly initialized: ['classifier.o

In [13]:
training_args = TrainingArguments(
    output_dir='./train_output2',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    #data_collator=data_collator,
)

In [14]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `EsmForSequenceClassification.forward` and have been ignored: text. If text are not expected by `EsmForSequenceClassification.forward`,  you can safely ignore this message.
/nfs/home/ababjac/base-venv/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 7200
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2250


Step,Training Loss
500,0.692000
1000,0.647300
1500,0.499200
2000,0.368400


Saving model checkpoint to ./train_output2/checkpoint-500
Configuration saved in ./train_output2/checkpoint-500/config.json
Model weights saved in ./train_output2/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./train_output2/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./train_output2/checkpoint-500/special_tokens_map.json
Saving model checkpoint to ./train_output2/checkpoint-1000
Configuration saved in ./train_output2/checkpoint-1000/config.json
Model weights saved in ./train_output2/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ./train_output2/checkpoint-1000/tokenizer_config.json
Special tokens file saved in ./train_output2/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to ./train_output2/checkpoint-1500
Configuration saved in ./train_output2/checkpoint-1500/config.json
Model weights saved in ./train_output2/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in ./train_output2/checkpoint-1500/token

TrainOutput(global_step=2250, training_loss=0.5262776217990451, metrics={'train_runtime': 121487.6953, 'train_samples_per_second': 0.296, 'train_steps_per_second': 0.019, 'total_flos': 4969130420173248.0, 'train_loss': 0.5262776217990451, 'epoch': 5.0})

In [15]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `EsmForSequenceClassification.forward` and have been ignored: text. If text are not expected by `EsmForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 800
  Batch size = 16


{'eval_loss': 0.5929588079452515,
 'eval_runtime': 640.5326,
 'eval_samples_per_second': 1.249,
 'eval_steps_per_second': 0.078,
 'epoch': 5.0}

In [16]:
trainer.predict(test_dataset=test_ds)

The following columns in the test set don't have a corresponding argument in `EsmForSequenceClassification.forward` and have been ignored: text. If text are not expected by `EsmForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 2000
  Batch size = 16


PredictionOutput(predictions=array([[-1.0249838 ,  1.0563256 ],
       [ 1.0486145 , -1.1835587 ],
       [ 0.889717  , -1.025582  ],
       ...,
       [ 0.19256382, -0.42237854],
       [-0.47040325,  0.6650371 ],
       [ 1.0476136 , -1.0625465 ]], dtype=float32), label_ids=array([0, 0, 0, ..., 1, 1, 0]), metrics={'test_loss': 0.6171581745147705, 'test_runtime': 1716.1217, 'test_samples_per_second': 1.165, 'test_steps_per_second': 0.073})

In [17]:
trainer.save_pretrained('./models/initial-esm')

AttributeError: 'Trainer' object has no attribute 'save_pretrained'